In [1]:
from tokenizer.BPE import tokenize, tokenizer
import pickle as pkl
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import torchaudio.datasets as datasets
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup
from transformer import TransformerLanguageModel
import wandb
from datetime import datetime

D:\PyCharmPrj\ML\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Tokenized dataset creation

In [2]:
current_dir = os.getcwd()

merges_path = os.path.join(current_dir, "tokenizer", "merges.pkl")
vocab_path = os.path.join(current_dir, "tokenizer", "vocabulary.pkl")

# Загрузка merges.pkl
with open(merges_path, "rb") as f:
    merges = pkl.load(f)
    print("Загрузка merges.pkl успешна")

# Загрузка vocab.pkl
with open(vocab_path, "rb") as f:
    vocab = pkl.load(f)
    print("Загрузка vocabulary.pkl успешна")

text = 'HELLO MY NAME IS BILL'
tokens = [vocab[0]] + tokenize(text, merges) + [vocab[1]]
#print(tokens)
#print(tokenizer.convert_tokens_to_string(tokens))

Загрузка merges.pkl успешна
Загрузка vocabulary.pkl успешна


In [3]:
wandb.init(project='TransformerLM')
config = {
    'learning_rate': 0.001,
    'batch_size': 64,
    'epochs': 64,
    'embedding_dim': 64,
    'dataset': "LibriSpeech dev-clean",
    'vocab_size': len(vocab),
}
wandb.config.update(config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: roman-kuznetsov (roman-kuznetsov-bmstu-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [4]:
token_to_id = {vocab[i]: i for i in range(len(vocab))}
id_to_token = {i: vocab[i] for i in range(len(vocab))}

data = datasets.LIBRISPEECH("../data", url="dev-clean", )
corpus = []
for i in range(2800):
    try:
        corpus.append(list(map(lambda x: token_to_id[x], [vocab[0]] + tokenize(data.__getitem__(i)[2], merges) + [vocab[1]])))
    except IndexError as err:
        break

max_length = max(len(seq) for seq in corpus)
print(max_length)
class TextDataset(Dataset):
    def __init__(self, data, max_len):
        self.data = data
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = torch.tensor(self.data[idx], dtype=torch.int16)
        sample = sample[:self.max_len]
        length = sample.shape[-1]
        padding = torch.ones((self.max_len - sample.shape[-1])) * 2
        sample = torch.cat((sample, padding), dim=0)
        return torch.tensor(sample, dtype=torch.float), length

dataset = TextDataset(corpus, max_length)

train_indices, val_indices = train_test_split(list(range(len(dataset))), test_size=0.2)

# Создание тренировочного и валидационного датасетов
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

# Создание DataLoader-ов для тренировочного и валидационного датасетов
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"])

294


In [11]:
print(token_to_id)

{'<|startoftext|>': 0, '<|endoftext|>': 1, '<|padding|>': 2, "'": 3, 'A': 4, 'B': 5, 'C': 6, 'D': 7, 'E': 8, 'F': 9, 'G': 10, 'H': 11, 'I': 12, 'J': 13, 'K': 14, 'L': 15, 'M': 16, 'N': 17, 'O': 18, 'P': 19, 'Q': 20, 'R': 21, 'S': 22, 'T': 23, 'U': 24, 'V': 25, 'W': 26, 'X': 27, 'Y': 28, 'Z': 29, 'Ġ': 30, 'ĠT': 31, 'HE': 32, 'ĠA': 33, 'IN': 34, 'ĠTHE': 35, 'ĠW': 36, 'ĠS': 37, 'ĠO': 38, 'RE': 39, 'ND': 40, 'ĠH': 41, 'ER': 42, 'ĠB': 43, 'ĠM': 44, 'OU': 45, 'IT': 46, 'ĠF': 47, 'IS': 48, 'ĠC': 49, 'AT': 50, 'ED': 51, 'ĠAND': 52, 'ĠOF': 53, 'EN': 54, 'ON': 55, 'ING': 56, 'ĠTO': 57, 'ĠP': 58, 'OR': 59, 'ES': 60, 'ĠD': 61, 'ĠTH': 62, 'ĠL': 63, 'AN': 64, 'AS': 65, 'ĠIN': 66, 'AR': 67, 'LL': 68, 'ĠN': 69, 'ĠHE': 70, 'ĠG': 71, 'AD': 72, 'LE': 73, 'OM': 74, 'ĠE': 75, 'ĠBE': 76, 'OT': 77, 'UT': 78, 'IC': 79, 'OW': 80, 'LY': 81, 'SE': 82, 'ĠI': 83, 'ST': 84, 'VE': 85, 'ĠWAS': 86, 'LD': 87, 'ĠWH': 88, 'GH': 89, 'ĠIT': 90, 'ĠTHAT': 91, 'ĠON': 92, 'ĠU': 93, 'ENT': 94, 'AL': 95, 'THE': 96, 'ID': 97, 'IM

In [10]:
for i, el in enumerate(val_loader):
    if i < 4:
        print(el)
    else:
        break

C:\Users\User\AppData\Local\Temp\ipykernel_78916\1362571734.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sample, dtype=torch.float), length


[tensor([[  0.,  96.,  38.,  ...,   2.,   2.,   2.],
        [  0.,   6.,  21.,  ...,   2.,   2.,   2.],
        [  0.,  22.,  11.,  ...,   2.,   2.,   2.],
        ...,
        [  0.,  46., 102.,  ...,   2.,   2.,   2.],
        [  0.,  55.,  35.,  ...,   2.,   2.,   2.],
        [  0.,  22.,  32.,  ...,   2.,   2.,   2.]]), tensor([ 41,  38, 152,  56, 212,  52,  52,  26, 109,  19,  64,  79,   9,  46,
         41,  46,  52,  26,  73,  17,  17,  41,  66,  52,  34,  84,  48,  17,
         77,  66,  16,   7,  20,  84,  32,  35,  73, 116,  59,  21,  60,  32,
         49,  65,  26,  96,  65, 115,  10,  37,  41,  43,  33,  80,  43,  65,
        163, 109,  69,  26,  63,  74,  47,  24])]
[tensor([[ 0., 16., 24.,  ...,  2.,  2.,  2.],
        [ 0., 26., 32.,  ...,  2.,  2.,  2.],
        [ 0., 12., 63.,  ...,  2.,  2.,  2.],
        ...,
        [ 0.,  7., 18.,  ...,  2.,  2.,  2.],
        [ 0., 45., 21.,  ...,  2.,  2.,  2.],
        [ 0., 46.,  3.,  ...,  2.,  2.,  2.]]), tensor([ 80,  21, 

In [5]:
def length_to_mask(data, lengths, dtype=None):
    batch_size, max_length = data.size(0), max(lengths)

    tgt_mask = torch.triu(torch.ones((max_length, max_length), device=device) == 1).transpose(0, 1)
    tgt_mask = tgt_mask.float().masked_fill(tgt_mask == 0, float('-inf')).masked_fill(tgt_mask == 1, float(0.0))

    # Создание key_padding_mask (2D)
    key_padding_mask = (torch.arange(max_length).expand(batch_size, max_length) >= lengths.unsqueeze(1))

    # Приведение к нужному типу, если указано
    if dtype is not None:
        key_padding_mask = key_padding_mask.to(dtype=dtype)

    return tgt_mask, key_padding_mask

In [6]:
# Параметры модели
vocab_size = config['vocab_size']
embedding_dim = config['embedding_dim']
dim_feedforward = 64
num_heads = 8
num_layers = 4
num_epochs = config['epochs']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TransformerLanguageModel(vocab_size, embedding_dim, num_heads, dim_feedforward, num_layers).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

total_steps = len(train_loader) * num_epochs

warmup_steps = 0.1 * total_steps

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

In [7]:
def train_one_epoch(epoch_index, model, training_loader, scheduler, optimizer, loss_fn):
    running_loss = 0.
    train_loss = 0.

    i = 0
    cnt_right_answers = 0
    cnt_answers = 0

    for data, lengths in training_loader:
        input_data = data.to(device)
        tgt_mask, tgt_key_padding_mask = length_to_mask(input_data, lengths)
        tgt_mask, tgt_key_padding_mask = tgt_mask.to(device), tgt_key_padding_mask.to(device)
        print(torch.max(lengths))

        if epoch_index != -1:
            optimizer.zero_grad()

        tgt_y = input_data[:, 1:]

        outputs, _ = model(input_data[:, :-1].to(int), tgt_mask=tgt_mask, lengths=lengths, tgt_key_padding_mask=tgt_key_padding_mask)

        answers = outputs.argmax(axis=-1)
        loss = loss_fn(outputs, tgt_y)
        if epoch_index != -1:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()
        train_loss += loss.item()
        if i % 10 == 9 and epoch_index != -1:
            last_loss = running_loss / 10 # loss per batch
            tb_x = epoch_index * len(training_loader) + i + 1
            wandb.log({'Loss': last_loss, "Train": tb_x})
            running_loss = 0.

        if i % 10 == 0 and scheduler is not None:
            wandb.log({'Scheduler LR': optimizer.param_groups[0]["lr"], "Train": tb_x})

        scheduler.step()

        cnt_answers += tgt_y.shape[0]
        cnt_right_answers += (answers == tgt_y).sum().item()

        i += 1

    train_loss = train_loss / len(training_loader)
    wandb.log({'Accuracy': cnt_right_answers / cnt_answers, "Train": epoch_index + 1})

    return train_loss


def validation(val_dataloader):
    val_loss = 0.0

    cnt_answers = 0
    cnt_right_answers = 0
    model.eval()
    with torch.no_grad():
        for i, vdata in enumerate(val_dataloader):
            input_data, lengths = vdata
            tgt_mask, tgt_key_padding_mask = length_to_mask(input_data, lengths)
            tgt_mask, tgt_key_padding_mask = tgt_mask.to(device), tgt_key_padding_mask.to(device)

            optimizer.zero_grad()
            input_data = input_data.to(device)
            labels = input_data[:, 1:].to(device)
            outputs, _ = model(input_data[:, :-1].to(int), tgt_mask=tgt_mask, lengths=lengths, tgt_key_padding_mask=tgt_key_padding_mask)

            vloss = criterion(outputs, labels)
            val_loss += vloss.item()
            answers = outputs.argmax(axis=-1)

            cnt_answers += labels.shape[0]
            cnt_right_answers += (answers == labels).sum().item()

            del input_data, labels, lengths

    val_loss = val_loss / len(val_dataloader)
    print("VAL LOSS =", val_loss)
    return val_loss, cnt_right_answers / cnt_answers

In [8]:
best_vloss = 1_000_000.

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

for epoch_number in range(num_epochs):
    print('EPOCH {}:'.format(epoch_number + 1))

    model.train()
    train_loss = train_one_epoch(
        epoch_index=epoch_number,
        model=model,
        training_loader=train_loader,
        optimizer=optimizer,
        scheduler=scheduler,
        loss_fn=criterion,)

    model.eval()
    val_loss, val_acc = validation(val_loader)

    wandb.log('Loss/valid', val_loss, epoch_number + 1)
    wandb.log('Accuracy/valid', val_acc, epoch_number + 1)
    if val_loss < best_vloss:
        best_vloss = val_loss
        model_path = os.path.join("../best_models/transformer", 'model_{}_{}'.format(epoch_number + 1, timestamp))
        torch.save(model.state_dict(), model_path)

wandb.finish()

EPOCH 1:


C:\Users\User\AppData\Local\Temp\ipykernel_78916\1362571734.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sample, dtype=torch.float), length


tensor(277)
torch.Size([277, 64, 64])


D:\PyCharmPrj\ML\.venv\Lib\site-packages\torch\nn\functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


RuntimeError: shape '[64, 512, 8]' is invalid for input of size 1134592